In [9]:
import time
import numpy as np
import cvxpy as cvx
import pandas as pd
import matplotlib.pyplot as plt
# cluster the load into different groups
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
from sklearn import manifold
from sklearn.metrics.pairwise import pairwise_distances

In [10]:
# from net_load data to get the cluster related variables
class cluster_net_load:
    def __init__(self, net_load):
        self.net_load = net_load
        
    def get_cluster(self):
        k = 16
        kmeans = KMeans(n_clusters=k, random_state = 42)
        kmeans = KMeans(n_clusters=k)
        y_pred = kmeans.fit_predict(self.net_load)
        centers = kmeans.cluster_centers_ 
        period_day_num = 0
        
        # get period
        cluster = y_pred[0]
        temp1 = []
        temp2 = []
        for i in range(len(y_pred)):
            if y_pred[i] == cluster:
                period_day_num = period_day_num+1
            else:
                temp1.append(period_day_num)
                temp2.append(cluster)
                cluster = y_pred[i]
                period_day_num = 1
        temp1.append(period_day_num)
        temp2.append(cluster)

        period = []
        period.append(temp1)
        period.append(temp2)
        period = np.array(period)
        period_data = pd.DataFrame(period)
        period_data.to_csv('data/period.csv', sep= ',', encoding='utf-8', header=False, index=False)
        
        # get cluster day_index and cluster day_num 
        data_cen = np.r_[self.net_load, centers]
        k = len(centers)
        closest_ind = np.arange(k)
        dist = pairwise_distances(data_cen, metric='euclidean')
        for i in range(k):
            closest_ind_tmp = np.where(dist[0:-k, -i-1] == np.min(dist[0:-k, -i-1]))
            closest_ind[i] = closest_ind_tmp[0][0]

        center_day = closest_ind+1
        # reverse
        center_day = center_day[::-1]
        
        cluster_day_num = []
        for cluster in range(0, k):
            day_cluster = np.argwhere(y_pred==cluster)
            cluster_day_num.append(len(day_cluster))
        cluster_day_num = np.array(cluster_day_num)
        
        cluster_day = []
        cluster_day.append(center_day)
        cluster_day.append(cluster_day_num)
        cluster_day = np.array(cluster_day)
        
        cluster_day_data = pd.DataFrame(cluster_day)
        cluster_day_data.to_csv('data/cluster_day.csv', sep= ',', encoding='utf-8', header=False, index=False)

In [11]:
df_load = pd.read_csv("data/day_load.csv", header=None)
load = df_load.values
cluster_load = cluster_net_load(10*load)
cluster_load.get_cluster()
print('cluster finished')

cluster finished
